# The "Perfect" Cipher - One Time Pads

After WWI died down, us Americans finally made a significant contribution to the field of cryptography. (Cheeky note: it must have been a Brit' like Singh who wrote that first sentence, because us Americans did some incredible code-making and code-breaking work in WWII). Anyway, the Americans found that one of the only real weaknesses of the Vigenére Cipher was that the key was used reptitively, allowing a cryptanalyst to look for patterns and then perform frequency analysis. But what if the key was really long. What if it was really, really long? The longer the key, the smaller sample that cryptanalysts would have for frequency analysis. What if we made the key the same length as the plaintext? Then there would be absolutely no patterns and the ciphertext would be perfectly secure.

Early One Time Pad Ciphers used a long key that was created from dictionary words. As time went on, the cryptanalysts discovered new ways to crack the Onetime Pads and cryptographers found new ways to secure them.

In [1]:
import vigenere
plaintext = "This is a secret message encrypted with a key of equal length"
longerkey = "GoofyPlutoMickeyMinnieDonaldDaisyHueyDeweyLouieScroogeClarice"

ciphertext = vigenere.encrypt(plaintext, longerkey)
print(ciphertext)

zvwx gh l mxqdmv wiqeitr mrfflpehg wqlf h eiw rj auslz fmryvy


In [2]:
sequences = vigenere.sequence_lists(plaintext, 4)
spans     = vigenere.sequence_span_lengths(sequences)
factors   = vigenere.sequence_length_factors(spans)
factors

{}

The previous example was too short to really be a good example, but if it had been longer, and if the key had been equally long, it would not have had any significant patterns to analyze. I didn't make the message any longer because it's hard to come up with a really long key. This was one of the problems with One Time Pads: how do you create the really long keys and how do you transfer them to a friend? You could use the text from a book or newspaper as the key, and many times, this was how they did it.

## The XOR Operator

The book introduces the original One Time Pad cipher which works on alphabetic text. But there's a much better, modern equivalent operates on binary data. This allows us to encrypt any generic type of data.

One Time Pad Ciphers started out as an extension of the Vigenére Cipher, but eventually cryptographers wanted to encrypt data that was not alphanumeric. They also wanted to use keys that had a wider range of values than the alphabet, with it's 26 possibilities (for English). The solution was to encrypt using the XOR operator on binary data.

XOR is the ultimate cryptographic operation. It operates at the bit level, so it can encrypt data and text. But what makes XOR so handy for encryption is that applying the same operation twice in a row reverts the data back to it's original form. You can see how this works in the truth table below.

|input|key|encoded|key|output|
|:---:|:-:|:-----:|:-:|:----:|
|0|0|0|0|0|
|0|1|1|1|0|
|1|0|1|0|1|
|1|1|0|1|1|

In [3]:
def one_time_pad(data, key):                                  # brings together parallel items from data and key
    paired  = [(d,k) for d,k in zip(data,key)]                #   (1st&1st, 2nd&2nd, 3rd&3rd, ...)
    crypted = [int.to_bytes(d^k, 1, 'big') for d,k in paired] # ^ operator performs the XOR
    crypted = b"".join(crypted)                               # join the items in the list back together into a string
    return crypted

message = b"The one time pad is a truely unbreakable cipher"
otp_key = b"zhdlcvklnvyioanervklfvn.lbgjnkwl;nwropvnjadsdfk"  # <-- random mashing of the keyboard

ciphertext1 = one_time_pad(message, otp_key)
print(ciphertext1)

plaintext1 = one_time_pad(ciphertext1, otp_key)
print(plaintext1)

b'.\x00\x01L\x0c\x18\x0eL\x1a\x1f\x14\x0cO\x11\x0f\x01R\x1f\x18L\x07V\x1a\\\x19\x07\x0b\x13N\x1e\x19\x0eI\x0b\x16\x19\x0e\x12\x1a\x0bJ\x02\r\x03\x0c\x03\x19'
b'The one time pad is a truely unbreakable cipher'


In the python language, XOR works on numbers but not on bit strings. We'll find it's easier to work with the data if we convert it from a string to a list of 1s and 0s. So let's write functions to convert back and forth.

#### Strings to Bit Arrays

In [4]:
test = b"otp"

print(test)
print(test[0], hex(test[0]), bin(test[0]))
print(test[1], hex(test[1]), bin(test[1]))
print(test[2], hex(test[2]), bin(test[2]))



b'otp'
111 0x6f 0b1101111
116 0x74 0b1110100
112 0x70 0b1110000


In [5]:
def bytes_to_bit_array(byte_string):
    bit_array = []
    for i in byte_string:
        # convert to binary, remove the '0b' notation from front, and retain any leading 0s to make 8 bits 
        # (python auto-truncates which we don't want)
        for j in bin(i)[2:].zfill(8): 
            bit_array.append(int(j))  # cast the binary string to int
    return bit_array

ex = b"A\xcd"
bytes_to_bit_array(ex)

[0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1]

#### Bit Arrays to Strings

Now let's code to go back the other way

In [6]:
test = [0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0]



In [7]:
def bit_array_to_bytes(bit_array):
    bytes = []
    for i in range(0,len(bit_array),8):
        # get 8 bits and convert them to string
        str_chunk = ''.join([str(b) for b in bit_array[i:i+8]])
        # convert str to binary (base2 int), then convert binary to char
        hex_chunk = chr(int(str_chunk,2))
        bytes.append(hex_chunk)
    
    return ''.join(bytes)

bit_array_to_bytes(test)

'otp'

#### Back to the OTP

In [8]:
def one_time_pad(data, key):
    
    bit_array = bytes_to_bit_array(data)
    key_array = bytes_to_bit_array(key)
    
    paired  = [(d,k) for d,k in zip(bit_array,key_array)]
    crypted = [d^k for d,k in paired]
    
    crypted = bit_array_to_bytes(crypted)
    return crypted

In [9]:
message = b"Not only is it impossible to break, but the ciphertext can be turned into every single possible plaintext"
otp_key = b"zhdlcvklnvyioanervklfvn.lbgjnkwl;nwropvnjadsdfkweiopwumCPmaIPCMKrtvne:Nvji;vnj2i05pnubiOJDio;3vniv)%VNJsl"

ciphertext = one_time_pad(message, otp_key)
print(ciphertext)

4LN
N	S M 1A+5c9> 
ES I^ KIC\	I?%7Y_NHL8:/


In [10]:
one_time_pad(ciphertext, otp_key)

TypeError: 'str' object cannot be interpreted as an integer

## One Time Pad and Plausible Deniability

Because the XOR operation can turn any bit into any other bit, depending on the key, the beauty of the One Time Pad is that the ciphertext can be turned into every possible plaintext! So, for the given ciphertext, you could have one key that turns it back into the real plaintext and a whole suite of others that turn it into fake plaintexts. For someone involved in a particularly sensitive and dangerous mission, this allows him to create a fake key that would decrypt the ciphertext to an alternate message. It would probably need to be a particularly damning plaintext, but not one that is quite as serious an offense as the real plaintext.

In [ ]:
#ciphertext ^ fakekey = result

## Homework

Create four One Time Pad XOR keys that decrypts the previous ciphertext to the four "fake" messages below.

```
message = b"Not only is it impossible to break, but the ciphertext can be turned into every single possible plaintext"
otp_key = b"zhdlcvklnvyioanervklfvn.lbgjnkwl;nwropvnjadsdfkweiopwumCPmaIPCMKrtvne:Nvji;vnj2i05pnubiOJDio;3vniv)%VNJsl"

ciphertext = one_time_pad(message, otp_key)
print(ciphertext)

output1 = b"This XOR operation allows you to turn any sequence of bits into any other sequence of bits, based only on"
output2 = b"It was the best of times, it was the worst of times, it was the age of wisdom, it was the age of foolishn"
output3 = b"ess, it was the epoch of belief, it was the epoch of incredulity, it was the season of Light, it was the "
output4 = b"season of Darkness, it was the spring of hope, it was the winter of despair, XXXXXXXXXXXXXXXXXXXXXXXXXXXX"
```

### Grading

This homework is worth 10 points.

### Turn-In

Submit the three One Time Pad XOR keys via Canvas. The format does not matter: you can put them in the submission text or attach them as files. Do not worry about any particular naming scheme.

In [1]:
import otp

In [4]:
message = b"Not only is it impossible to break, but the ciphertext can be turned into every single possible plaintext"
otp_key = b"zhdlcvklnvyioanervklfvn.lbgjnkwl;nwropvnjadsdfkweiopwumCPmaIPCMKrtvne:Nvji;vnj2i05pnubiOJDio;3vniv)%VNJsl"
ciphertext = otp.crypt(message, otp_key)
print(ciphertext)

output1 = b"This XOR operation allows you to turn any sequence of bits into any other sequence of bits, based only on"
output2 = b"It was the best of times, it was the worst of times, it was the age of wisdom, it was the age of foolishn"
output3 = b"ess, it was the epoch of belief, it was the epoch of incredulity, it was the season of Light, it was the "
output4 = b"season of Darkness, it was the spring of hope, it was the winter of despair, XXXXXXXXXXXXXXXXXXXXXXXXXXXX"

fakekey = otp.crypt(message, output1)
print(fakekey)
print(fakekey.decode("ascii"))
# print(otp.crypt(message, fakekey))

b'4\x07\x10L\x0c\x18\x07\x15N\x1f\nI\x06\x15N\x0c\x1f\x06\x04\x1f\x15\x1f\x0cB\tB\x13\x05N\t\x05\tZ\x05[R\r\x05\x02N\x1e\t\x01S\x07\x0f\x1b\x1f\x00\x1b\x1b\x15\x0f\x01M 1\x03A+5c9>\x00\x1a\x13\nES \x02\x05I^\x00\x0b\x18KIC\\\x1e\t\x19\x07I?%7\x1a\x06Y_\x13N\x19\x1aHL8:/\x0b\x18'
b'\x1a\x07\x1dSO6#+\x00\x06\x03E\x1b\x15T\x00\x02\x1eO\x12\x1f\x05\r\x1b\x16\x00\r\x00UB\x06\nA\x1fYR\x0cU\x15N\rH\x16E\x12\x1c\x15\x06\x06\x17T\n\x1eTB\n\x15\x1d\x00\x0b\x0bT\x1bU\x13\x00\x1cDO\x1d\x06\x11\x1d\x00\x16\x13\x14\x07\x1cN\x10\x0cN\x08\nEB\x19\x1b\x00_I\x00\r\x16E\x14L\x0e\x07\x02\rE\x17\x1a'
 UBO6#+ ET O
HET
TB
 TU DO N
EL 
